## Chapter 12

### Chain of responsibility

In [1]:
class InvoiceHandler(object):
    def __init__(self):
        self.nextAction = None
    
    def handle(self,printinvoice):
        self.nextAction.handle(printinvoice)

class InputState(object):
    stateNY = ['NYC','NY','New York','new york']
    stateCA = ['CA', 'California', 'california']
    
class PrintInvoice(object):
    def __init__(self,state):
        self.state = state
        self.header = 'State specific Sales tax is applicable for the state of ' + self.state

class NYCHandler(InvoiceHandler):
    def generateInvoice(self, header, state):
        product = 'WashingMachine'
        pricebeforetax = 450 + (450 * 0.19)
        taxRate = 0.4
        localRate = 0.055
        tax = pricebeforetax * (taxRate + localRate)
        finalsellingprice = pricebeforetax + tax
        print('**************ABC Megamart*****************')
        print('***********------------------**************')
        print(header)
        print('Product: ', product)
        print('Tax: ', tax)
        print('Total Price: ', finalsellingprice)
        print('***********------------------**************') 
        
    def handle(self,printinvoice):
        if printinvoice.state in InputState.stateNY:
            self.generateInvoice(printinvoice.header, printinvoice.state)
        
        else:
            super(NYCHandler, self).handle(printinvoice)
            
class CAHandler(InvoiceHandler):
    def generateInvoice(self, header, state):
        product = 'WashingMachine'
        pricebeforetax = 480 + (480 * 0.14)
        taxRate = 0.35
        localRate = 0.077
        tax = pricebeforetax * (taxRate + localRate)
        finalsellingprice = pricebeforetax + tax
        print('**************ABC Megamart*****************')
        print('***********------------------**************')
        print(header)
        print('Product: ', product)
        print('Tax: ', tax)
        print('Total Price: ', finalsellingprice)
        print('***********------------------**************') 
        
    def handle(self,printinvoice):
        if printinvoice.state in InputState.stateCA:
            self.generateInvoice(printinvoice.header, printinvoice.state)
        
        else:
            super(CAHandler, self).handle(printinvoice)

class ExceptionHandler(InvoiceHandler):
    def handle(self,printinvoice):
        print("No branches in the state")  
   

In [2]:
def invoiceRequestor(state):
    invoice = PrintInvoice(state)
    nychandler = NYCHandler()
    cahandler = CAHandler()

    nychandler.nextAction = cahandler
    cahandler.nextAction = ExceptionHandler()
    nychandler.handle(invoice)

In [3]:
invoiceRequestor('CA')

**************ABC Megamart*****************
***********------------------**************
State specific Sales tax is applicable for the state of CA
Product:  WashingMachine
Tax:  233.6544
Total Price:  780.8544
***********------------------**************


In [4]:
invoiceRequestor('NYC')

**************ABC Megamart*****************
***********------------------**************
State specific Sales tax is applicable for the state of NYC
Product:  WashingMachine
Tax:  243.6525
Total Price:  779.1525
***********------------------**************


In [5]:
invoiceRequestor('TEXAS')

No branches in the state


### Command

In [6]:
from abc import ABC, abstractmethod

class Billing:
    sales = {'purchasePrice': 450,
              'profitMargin': 0.19,
              'taxRate': 0.4,
              'discountRate': 0.10
              }        

    @abstractmethod
    def applyDiscount(self):
        pass
    
    @abstractmethod
    def removeDiscount(self):
        pass

In [7]:
class DiscountedBilling(Billing):

    def applyDiscount(self):
        sales = self.sales
        pricebeforetax = sales['purchasePrice'] + sales['purchasePrice'] * sales['profitMargin']
        finalsellingprice = pricebeforetax + (pricebeforetax * sales['taxRate'])
        sales['sellingPrice'] = finalsellingprice
        discountedPrice = sales['sellingPrice'] * (1 - sales['discountRate'])
        return discountedPrice

In [8]:
class ActualBilling(Billing):

    def removeDiscount(self):
        sales = self.sales
        pricebeforetax = sales['purchasePrice'] + sales['purchasePrice'] * sales['profitMargin']
        actualprice = pricebeforetax + (pricebeforetax * sales['taxRate'])
        return actualprice

In [9]:
class ExecuteBilling:
    @abstractmethod
    def execDiscount(self):
        pass

    @abstractmethod
    def revokeDiscount(self):
        pass

In [10]:
class ExecuteDiscountedBilling(ExecuteBilling):
    def __init__(self, instance):
        self.instance = instance
        
    def execDiscount(self):
        print('Discount applied...')
        return self.instance.applyDiscount()
        
    def revokeDiscount(self, revokeInstance):
        revokeInstance.reset(ExecuteActualBilling(ActualBilling()))
        return revokeInstance.runcalc()

In [11]:
class ExecuteActualBilling(ExecuteBilling):
    def __init__(self, instance):
        self.instance = instance
        
    def execDiscount(self):
        print('Discount removed...')
        return self.instance.removeDiscount()
        
    def revokeDiscount(self, revokeInstance):
        revokeInstance.reset(ExecuteDiscountedBilling(DiscountedBilling()))
        return revokeInstance.runcalc()

In [12]:
class RequestAction:
    def __init__(self, action):
        self.action = action

    def reset(self, action):
        print("Resetting command...")
        self.action = action

    def runcalc(self):
        return self.action.execDiscount()
    
    def revert(self):
        print("Reverting the previous action...")
        return self.action.revokeDiscount(self)

In [13]:
class Tester:
    def __init__(self):
        billing = Billing()

        discount = ExecuteDiscountedBilling(DiscountedBilling())
        actual = ExecuteActualBilling(ActualBilling())
        requestor = RequestAction(discount)  

        print(requestor.runcalc())

        requestor.reset(actual)

        print(requestor.runcalc())

        print(requestor.revert())

        print(requestor.revert()) 

In [14]:
Tester()

Discount applied...
674.73
Resetting command...
Discount removed...
749.7
Reverting the previous action...
Resetting command...
Discount applied...
674.73
Reverting the previous action...
Resetting command...
Discount removed...
749.7


### Strategy pattern

In [15]:
class SuperMarket():
    
    def __init__(self,STRATEGY, items, name, scan, units, tax, itemtype = None):
        self.STRATEGY = STRATEGY
        self.items = items
        self.name = name
        self.scan = scan
        self.units = units
        self.tax = tax
        self.itemtype = itemtype
       
    def returnCart(self):
        cartItems = []
        for i in self.items:
            cartItems.append(i)
        return cartItems
    
    def gotoCounter(self):
        countername = self.name
        return countername

    def scanBarCode(self):
        codes = []
        for i in self.scan:
            codes.append(i)
        return codes
          
    def addBilling(self):
        self.codes = self.scanBarCode()
        pricetag = []
        for i in self.units:
            pricetag.append(i)
        bill = dict(zip(self.codes, pricetag))
        return bill
        
    def addTax(self):
        taxed = []
        for i in self.tax:
            taxed.append(i)
        return taxed
    
    def calcBill(self):
        bill = self.addBilling()
        items = []
        cartItems = self.returnCart()
        calcbill = []
        taxes = self.addTax()
        for item,tax in zip(bill.items(),taxes):
            items.append(item[1])
            calcbill.append(item[1] + item[1]*tax)
        finalbill = dict(zip(cartItems, calcbill))
        return finalbill
        
    def printInvoice(self):
        finalbill = self.calcBill()
        final_total = sum(finalbill.values())
        print('**************ABC Megamart*****************')
        print('***********------------------**************')
        print('Counter Name: ', self.gotoCounter())
        for item,price in finalbill.items():
            print(item,": ", price)
        print('Total:',final_total)
        print('***********------------------**************')
        print('***************PAID************************')
         
    def pipeline_template(self):
        self.returnCart()
        self.gotoCounter()
        self.STRATEGY.redirectCounter()
        self.scanBarCode()
        self.addBilling()
        self.addTax()
        self.calcBill()
        self.printInvoice()

In [16]:
class VegeCounter():
    def redirectCounter():
        print("**************Move to Vege Counter**************")

In [17]:
class ElectronicsCounter():
    def redirectCounter():
        print("**************Move to Electronics Counter**************")

In [18]:
def run_pipeline(domain = SuperMarket):
    domain.pipeline_template()

In [19]:
run_pipeline(SuperMarket(STRATEGY = VegeCounter,
                           items = ['Onions','Tomatoes','Cabbage','Beetroot'],
                         name = ['Vegetable Counter'],
                         scan = [113323,3434332,2131243,2332783],
                         units = [10,15,12,14],
                         tax = [0.04,0.03,0.035,0.025],
                         itemtype = ['Vegetables'],
                         ))

**************Move to Vege Counter**************
**************ABC Megamart*****************
***********------------------**************
Counter Name:  ['Vegetable Counter']
Onions :  10.4
Tomatoes :  15.45
Cabbage :  12.42
Beetroot :  14.35
Total: 52.620000000000005
***********------------------**************
***************PAID************************


In [20]:
run_pipeline(SuperMarket(STRATEGY = ElectronicsCounter,
                         items = ['television','keyboard','mouse'],
                                name = ['Electronics Counter'],
                                scan = [113323,3434332,2131243],
                                units = [100,16,14],
                                tax = [0.04,0.03,0.035],
                                itemtype = ['Electronics'],
                                ))

**************Move to Electronics Counter**************
**************ABC Megamart*****************
***********------------------**************
Counter Name:  ['Electronics Counter']
television :  104.0
keyboard :  16.48
mouse :  14.49
Total: 134.97
***********------------------**************
***************PAID************************


### These are all the examples covered in this chapter.